In [1]:
import numpy as np
import pandas as pd

import pymc3 as pm

from pyddcurves import utils
from pyddcurves import models as dd

In [14]:
basedir = '../data'
datafile1 = '%s/Shell_Density_KP150_phs1'%basedir
datafile2 = '%s/Shell_Density_KP150_phs2'%basedir

z_std = 100.
rho_std = 1.5
rho_mu = 1024.

kdvtime = pd.date_range('2016-05-01','2017-05-02',freq='D').values

###
kdvtime1 = pd.date_range('2016-05-01','2016-10-31',freq='D').values
kdvtime2 = pd.date_range('2016-11-01','2017-05-02',freq='D').values


In [22]:
print('Loading the input data...')
rho1 = utils.read_density_csv('%s.csv'%datafile1)
rho2 = utils.read_density_csv('%s.csv'%datafile2)

# Interpolate on a constant time grid
rho1i = rho1.interp({'time':kdvtime1}, method='nearest',kwargs={"fill_value": "extrapolate"})
rho2i = rho2.interp({'time':kdvtime2}, method='nearest',kwargs={"fill_value": "extrapolate"})

ntimeavg = 1 


## First deployment
rho = rho1i
nt,nz = rho.shape

depths_2d = rho.depth.values[np.newaxis,:].repeat(nt, axis=0)

obsdata1 = utils.density_to_obsdict(rho.values, depths_2d, rho.time.values, ntimeavg, z_std, rho_mu, rho_std)

# Second deployment
rho = rho2i
nt,nz = rho.shape

depths_2d = rho.depth.values[np.newaxis,:].repeat(nt, axis=0)

obsdata2 = utils.density_to_obsdict(rho.values, depths_2d, rho.time.values, ntimeavg, z_std, rho_mu, rho_std)

obsdata = utils.merge_obs_dicts(obsdata1, obsdata2)

Loading the input data...


In [23]:
obsdata

{'N': 3853,
 'n_times': 367,
 'rho': array([ 1.464     ,  1.344     ,  1.22333333, ..., -1.43733333,
        -1.58133333, -1.61133333]),
 'z': array([-2.505, -2.275, -2.03 , ..., -0.632, -0.396, -0.177]),
 'timeidx': array([  0,   0,   0, ..., 366, 366, 366]),
 'rho_std': 1.5,
 'rho_mu': 1024.0,
 'z_std': 100.0,
 'time': array([1462060800000000000, 1462147200000000000, 1462233600000000000,
        1462320000000000000, 1462406400000000000, 1462492800000000000,
        1462579200000000000, 1462665600000000000, 1462752000000000000,
        1462838400000000000, 1462924800000000000, 1463011200000000000,
        1463097600000000000, 1463184000000000000, 1463270400000000000,
        1463356800000000000, 1463443200000000000, 1463529600000000000,
        1463616000000000000, 1463702400000000000, 1463788800000000000,
        1463875200000000000, 1463961600000000000, 1464048000000000000,
        1464134400000000000, 1464220800000000000, 1464307200000000000,
        1464393600000000000, 1464480000

In [21]:
rho1.interp({'time':kdvtime1}, method='nearest',kwargs={"fill_value": "extrapolate"})

<xarray.DataArray (time: 184, depth: 10)>
array([[1026.196, 1026.016, 1025.835, ..., 1023.245, 1022.395, 1021.149],
       [1026.196, 1026.016, 1025.835, ..., 1023.245, 1022.395, 1021.149],
       [1026.196, 1026.016, 1025.835, ..., 1023.245, 1022.395, 1021.149],
       ...,
       [1025.875, 1025.566, 1025.388, ..., 1022.42 , 1022.019, 1021.685],
       [1025.866, 1025.582, 1025.39 , ..., 1022.458, 1022.048, 1021.687],
       [1025.858, 1025.591, 1025.363, ..., 1022.483, 1022.063, 1021.647]])
Coordinates:
  * depth    (depth) float64 -250.5 -227.5 -203.0 -178.5 ... -80.5 -56.1 -31.7
  * time     (time) datetime64[ns] 2016-05-01 2016-05-02 ... 2016-10-31

In [20]:
rho1.interp?

Signature:
rho1.interp(
    coords: Mapping[Hashable, Any] = None,
    method: str = 'linear',
    assume_sorted: bool = False,
    kwargs: Mapping[str, Any] = None,
    **coords_kwargs: Any,
) -> 'DataArray'
Docstring:
Multidimensional interpolation of variables.

Parameters
----------
coords : dict, optional
    Mapping from dimension names to the new coordinates.
    New coordinate can be an scalar, array-like or DataArray.
    If DataArrays are passed as new coordinates, their dimensions are
    used for the broadcasting. Missing values are skipped.
method : str, default: "linear"
    The method used to interpolate. Choose from

    - {"linear", "nearest"} for multidimensional array,
    - {"linear", "nearest", "zero", "slinear", "quadratic", "cubic"} for 1-dimensional array.
assume_sorted : bool, optional
    If False, values of x can be in any order and they are sorted
    first. If True, x has to be an array of monotonically increasing
    values.
kwargs : dict
    Additional ke